# Redesign of nations and how they are created. 

Events are happening all the time. Several events happen before the Character quest begins. 

In [1]:
import pandas as pd
import numpy as np
import os

import os
import sys
import django
sys.path.append('../..')
#Loading my project settings from prodweb. This allows me to load and query models. 
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'prodweb.settings')
django.setup()

#I'm mapping to the actual files in the repo so that I can also use this to troubleshoot
import sys,pickle
sys.path.append('../')
from lib.builders import people,towns

with (open("../pickles/billmanhworld.pkl", 'rb')) as pickle_file:
    world = pickle.load(pickle_file)

In [2]:
from sklearn.cluster import KMeans

def cluster_nations(world):
    cities = world.df_features[world.df_features['terrain']=='town']
    k = KMeans(init='k-means++', n_clusters=world.culture.n_nations, n_init=10)
    k.fit(cities[['x','y']])
    labelNations(k,world)

def labelNations(k,world):
    world.nations = []
    for n in np.unique(k.labels_):
        nation = Nation(world,k=n)
        world.nations.append(nation)
        

class Nation:
    def __init__(self,world,**kwargs):
        self.name = self.name_nation(world)
        if kwargs.get("k",None):
            #Kmeans is the default (when the world is created)
            #Requires the integer value used when creating the world. 
            self.cast_nation(world,kwargs.get("k"))
            self.towns = self.addTowns(world)
        
        self.diplomacy = self.add_diplomacy(world)
    
    def cast_nation(self,world,k):
        world.df_features.loc[world.df_features['nation number']==k] = self.name
        
    def add_towns(self,world):
        towns = [town for town in world.towns 
                 if town.diplomacy.get('nation','none')==self.name] 
        return towns
    
    def name_nation(self,world):
        """
        lables a list of nations from a k-means cluster
        """
        nation_name = world.culture.townNameGenerator()
        return nation_name

    def __repr__(self):
        return f"Nation of {self.name}"
       
    def addTowns(self,world):
        towns = [town for town in world.towns 
                 if town.diplomacy.get('nation','none')==self.name]
        self.towns = towns
        
    def get_capitol(self):
        c = [t for t in self.towns if t.type == 'capitol'][0]
        return c
    
    def get_random_town(self):
        return np.random.choice(self.towns)

    def getRuler_str(self):
        return self.ruler.name
    
    def getRuler(self):
        return self.ruler
    
    def add_diplomacy(self,world):
        # if the world already has nations, then this is a new nation. 
        diplomacy = {}
        otherNations = [n for n in world.nations if n.name != self.name]
        for o in otherNations:
            diplomacy[o] = {'favor':.8}
        return diplomacy
    
    def get_deplomacy_df(self):
        d = pd.DataFrame(self.diplomacy).T.reset_index(drop=False)
        d['nation'] = self.name
        d.columns = ['neighbor','favor','stance','nation']
        return d[['nation','neighbor','favor','stance']]
            
    def appointRuler(self,person):
        t = self.getCapitol_str(self.towns)
        self.ruler = person(f'ruler of {self.name}',t)
        
nation = Nation(world,k=0)
print(nation)

Nation of Ragefort


In [3]:
cluster_nations(world)

In [4]:
world.nations

[Nation of Brinelight,
 Nation of Warkeep,
 Nation of Lordscombe,
 Nation of Gleamlair,
 Nation of Clearmain,
 Nation of Heartforth,
 Nation of Stenchtown,
 Nation of Solrun]

In [5]:
[[town for town in nation.towns] for nation in world.nations]

AttributeError: 'Nation' object has no attribute 'towns'

In [7]:
n = world.nations[1]

In [8]:
n.towns